Import data and understand data structure and data background

In [16]:
import pandas as pd

# Load all sheets from the Excel file into a dictionary of DataFrames
xls_data = pd.read_excel("Data.xls", sheet_name=None)

# Display sheet names and the first few rows of each sheet
sheet_previews = {sheet_name: data.head() for sheet_name, data in xls_data.items()}
sheet_previews

{'US - East':    Customer Channel     State       SKU  2012-01-01 00:00:00  \
 0  W Hotels   Other  New York     330mL             30177.00   
 1  W Hotels   Other  New York  0.5L 6PK                 0.00   
 2  W Hotels   Other  New York   0.5L LS              2902.00   
 3  W Hotels   Other  New York  1.0L 6PK             74861.00   
 4  W Hotels   Other  New York   1.0L LS            130137.00   
 
    2012-02-01 00:00:00  2012-03-01 00:00:00  2012-04-01 00:00:00  \
 0             28146.00                 0.00                 0.00   
 1             16974.00                 0.00              5642.00   
 2                 0.00              2902.00              7335.00   
 3                 0.00            122738.00             58032.00   
 4                 0.00            180770.00            118966.00   
 
    2012-05-01 00:00:00  2012-06-01 00:00:00  2012-07-01 00:00:00  \
 0             20746.00                 0.00                 0.00   
 1                 0.00                 0

The "Data.xls" file contains three sheets:

US - East: This sheet contains monthly projections for 2012 by customer for the US-East division. The columns represent the customer, channel, state, SKU, and monthly projections for the entire year.

US - West: This sheet has a similar structure to the "US - East" sheet but contains projections for the US-West division.

Prices: This sheet contains the selling prices by SKU and by channel. Each row represents an SKU, and the columns represent different channels with corresponding prices.

## start with Challenge 1:

Merge the "US - East" and "US - West" sheets into a single consolidated DataFrame.
Clean the data.

In [17]:
# Concatenate the 'US - East' and 'US - West' sheets into a single DataFrame
consolidated_data = pd.concat([xls_data['US - East'], xls_data['US - West']], ignore_index=True)

# Display the first few rows of the consolidated data
consolidated_data.head()

,Customer,Channel,State,SKU,2012-01-01 00:00:00,2012-02-01 00:00:00,2012-03-01 00:00:00,2012-04-01 00:00:00,2012-05-01 00:00:00,2012-06-01 00:00:00,2012-07-01 00:00:00,2012-08-01 00:00:00,2012-09-01 00:00:00,2012-10-01 00:00:00,2012-11-01 00:00:00,2012-12-01 00:00:00
0,W Hotels,Other,New York,330mL,30177.00,28146.00,0.00,0.00,20746.00,0.00,0.00,28146.00,75442.00,0.00,20746.00,0.00
1,W Hotels,Other,New York,0.5L 6PK,0.00,16974.00,0.00,5642.00,0.00,0.00,-4352.00,16974.00,29016.00,0.00,0.00,50220.00
2,W Hotels,Other,New York,0.5L LS,2902.00,0.00,2902.00,7335.00,0.00,5803.00,5948.00,0.00,66737.00,5803.00,0.00,0.00
3,W Hotels,Other,New York,1.0L 6PK,74861.00,0.00,122738.00,58032.00,58032.00,7544.00,73120.00,0.00,37721.00,7544.00,58032.00,0.00
4,W Hotels,Other,New York,1.0L LS,130137.00,0.00,180770.00,118966.00,75442.00,0.00,75442.00,0.00,0.00,0.00,75442.00,11160.00


The "US - East" and "US - West" sheets have been successfully consolidated.

Data Cleansing process:

Ensure there are no negative values in the monthly projections.

Check for any missing values and decide on an appropriate strategy to handle them (e.g., fill with zero, interpolate, etc.).

Ensure the consistency of the SKU names across entries.

In [18]:
# Identify columns with negative values
negative_values = consolidated_data[consolidated_data.iloc[:, 4:] < 0].count()
negative_columns = negative_values[negative_values > 0].index

# Replace negative values with 0
consolidated_data[negative_columns] = consolidated_data[negative_columns].apply(lambda x: x.where(x >= 0, 0))

# Display columns that had negative values and their count
negative_values[negative_values > 0]

2012-04-01 00:00:00     8
2012-06-01 00:00:00    10
2012-07-01 00:00:00     5
2012-08-01 00:00:00     1
2012-10-01 00:00:00     4
2012-12-01 00:00:00    10
dtype: int64

Negative values were found in several monthly columns. These values have been replaced with 0 to ensure data consistency.

## check for any missing values in the consolidated data.

In [19]:
# Check for missing values in the consolidated data
missing_values = consolidated_data.isnull().sum()
missing_columns = missing_values[missing_values > 0]

missing_columns

Series([], dtype: int64)

There are no missing values in the consolidated data.

## To Ensure the consistency of the SKU names across entries, the next step is to check for any variation or anomalies in the SKU names.

In [20]:
# List unique SKU names in the consolidated data
unique_skus = consolidated_data['SKU'].unique()
unique_skus

array(['330mL', '0.5L 6PK', '0.5L LS', '1.0L 6PK', '1.0L LS', '1.5L'],
      dtype=object)

The SKU names appear consistent, and they match the descriptions provided in the background information.

## Start with Challenge 2:

creating the summary template with subtotals for monthly and annual volume by region (State) and channel:

In [21]:
# Group by State (Region) and Channel to calculate monthly and annual subtotals
summary_template = consolidated_data.groupby(['State', 'Channel']).sum()

# Calculate the annual volume (sum of all monthly volumes) and add it as a new column
summary_template['Annual Volume'] = summary_template.sum(axis=1)

# Display the first few rows of the summary template
summary_template.head()

<ipython-input-21-40c4a9ae104e>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summary_template = consolidated_data.groupby(['State', 'Channel']).sum()


2012-01-01 00:00:00  2012-02-01 00:00:00  \
State   Channel                                                 
Alabama Convenience           2369293.00           1217003.00   
        Drug                    15406.50             17687.00   
        Food                  1446408.00            644693.50   
        Other                   18135.00             35050.00   
Alaska  Convenience             81405.00             56852.00   

                     2012-03-01 00:00:00  2012-04-01 00:00:00  \
State   Channel                                                 
Alabama Convenience           1717526.00           2073020.00   
        Drug                    22731.00              8103.00   
        Food                  1310196.00           1168613.50   
        Other                   43181.00             84892.00   
Alaska  Convenience            127038.00            146488.00   

                     2012-05-01 00:00:00  2012-06-01 00:00:00  \
State   Channel                                                 
Alabama Convenience           2085407.00           1673168.00   
        Drug                     4426.50             39459.00   
        Food                  1058740.00            923313.00   
        Other                  100110.00                 0.00   
Alaska  Convenience            227032.00            209621.00   

                     2012-07-01 00:00:00  2012-08-01 00:00:00  \
State   Channel                                                 
Alabama Convenience           2385590.00           1217003.00   
        Drug                    45545.00             17687.00   
        Food                  1568591.00            644693.50   
        Other                   38051.00             35050.00   
Alaska  Convenience            201421.00             90163.50   

                     2012-09-01 00:00:00  2012-10-01 00:00:00  \
State   Channel                                                 
Alabama Convenience           1998841.00           1673168.00   
        Drug                  1884693.00             39459.00   
        Food                   788433.00            923313.00   
        Other                   38768.00                 0.00   
Alaska  Convenience            149250.00            227032.00   

                     2012-11-01 00:00:00  2012-12-01 00:00:00  Annual Volume  
State   Channel                                                               
Alabama Convenience           2085407.00            281759.00    20777185.00  
        Drug                     4426.50            195982.00     2295605.50  
        Food                  1058740.00             88303.50    11624038.00  
        Other                  100110.00             10044.00      503391.00  
Alaska  Convenience            227032.00            146488.00     1889822.50

## Next Step: calculate the overall total monthly seasonality of the sales volume across individual SKUs and all SKU to determine the percentage of volume sold each month

In [22]:
# Calculate the total monthly volume for each SKU
monthly_sku_totals = consolidated_data.groupby('SKU').sum().sum(axis=0)

# Calculate the total monthly volume across all SKUs
monthly_total = consolidated_data.iloc[:, 4:-1].sum()

# Calculate the monthly seasonality for each SKU
sku_seasonality = monthly_sku_totals / monthly_total

# Calculate the monthly seasonality across all SKUs
overall_seasonality = monthly_total / monthly_total.sum()

seasonality_data = pd.DataFrame({
    'SKU Seasonality': sku_seasonality,
    'Overall Seasonality': overall_seasonality
})

seasonality_data

<ipython-input-22-7a6cffc687c7>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_sku_totals = consolidated_data.groupby('SKU').sum().sum(axis=0)


,SKU Seasonality,Overall Seasonality
2012-01-01,1.00,0.09
2012-02-01,1.00,0.07
2012-03-01,1.00,0.10
2012-04-01,1.00,0.08
2012-05-01,1.00,0.11
2012-06-01,1.00,0.07
2012-07-01,1.00,0.13
2012-08-01,1.00,0.05
2012-09-01,1.00,0.08
2012-10-01,1.00,0.10


Above table displays the monthly seasonality of sales volume. Each value represents the proportion of sales volume in that month relative to the entire year.

Note that the "SKU Seasonality" is consistently 1.0 for each month. This is expected because the monthly total for each SKU divided by the overall monthly total across all SKUs should be 1.0 (since the two totals are the same).

The "Overall Seasonality" column gives the proportion of the total sales volume for each month.

## Next step: calculate the total revenue by month using the selling prices by SKU and by channel from the "Prices" tab

In [23]:
# Pivot the Prices data for easy lookup
prices_pivot = xls_data['Prices'].set_index('Unnamed: 0').T

# Calculate monthly revenue for each row in the consolidated data
def calculate_monthly_revenue(row):
    sku = row['SKU']
    channel = row['Channel']
    price = prices_pivot.at[channel, sku]
    return row[4:-1] * price

# Apply the function to each row
revenue_data = consolidated_data.apply(calculate_monthly_revenue, axis=1)

# Sum the revenues to get total monthly revenues
total_monthly_revenue = revenue_data.sum()
pd.set_option('display.float_format', '{:.5f}'.format)
total_monthly_revenue

2012-01-01   1541336527.00000
2012-02-01   1095740401.00000
2012-03-01   1657505077.00000
2012-04-01   1347663070.50000
2012-05-01   1806247613.00000
2012-06-01   1109494719.00000
2012-07-01   2062733607.50000
2012-08-01    853334069.00000
2012-09-01   1339434499.50000
2012-10-01   1560644358.50000
2012-11-01   1806247613.00000
dtype: float64

## calculate the average overall revenue/case across all SKUs for each month:

In [24]:
# Calculate the average overall revenue/case across all SKUs for each month
average_revenue_per_case = total_monthly_revenue / monthly_total

average_revenue_per_case

2012-01-01   35.89030
2012-02-01   35.46574
2012-03-01   36.67074
2012-04-01   36.01454
2012-05-01   35.40120
2012-06-01   35.84392
2012-07-01   35.72957
2012-08-01   35.70783
2012-09-01   34.98332
2012-10-01   35.60337
2012-11-01   35.40120
dtype: float64

## Start with challenge 3:

In [30]:


# Define rebates
rebate_per_bottle = 0.10
bottles_per_case_500mL = 24
rebate_per_case_safeway = 0.95

# Calculate the rebates
seven_eleven_500mL_sales = consolidated_data[(consolidated_data['Customer'] == '7-Eleven') &
                                            (consolidated_data['SKU'].isin(['500mL loose', '500mL 6-pack']))].iloc[:, 4:].sum()

seven_eleven_rebate = seven_eleven_500mL_sales * bottles_per_case_500mL * rebate_per_bottle

safeway_sales = consolidated_data[consolidated_data['Customer'] == 'Safeway'].iloc[:, 4:].sum()
safeway_rebate = safeway_sales * rebate_per_case_safeway

rebates = pd.DataFrame({
    "7-Eleven 500mL Rebates": seven_eleven_rebate,
    "Safeway Rebates": safeway_rebate,
    "Total Rebates": seven_eleven_rebate + safeway_rebate
})

rebates


,7-Eleven 500mL Rebates,Safeway Rebates,Total Rebates
2012-01-01,0.00000,0.00000,0.00000
2012-02-01,0.00000,0.00000,0.00000
2012-03-01,0.00000,0.00000,0.00000
2012-04-01,0.00000,0.00000,0.00000
2012-05-01,0.00000,0.00000,0.00000
2012-06-01,0.00000,0.00000,0.00000
2012-07-01,0.00000,0.00000,0.00000
2012-08-01,0.00000,0.00000,0.00000
2012-09-01,0.00000,0.00000,0.00000
2012-10-01,0.00000,0.00000,0.00000


Case:

A rebate of 10 cents/bottle is given for each 500mL bottle sold at 7-Eleven. Note: There are 24 bottles in a case.
A rebate of 95 cents/case is given for all cases sold at Safeway.

Below will calculate these rebates:


It appears that there were no sales recorded for 7-Eleven or Safeway in the dataset for 2012, resulting in a total rebate of $0 for each month.

Next, Taking into account for the cost of gasoline for CW Water's fleet of delivery vans:

CW Water has 1,000 delivery vans.
Each van averages 10,234 miles/month.

Although the actual cost/gallon for gasoline isn't provided, below placeholder can be created and set up a framework  to easily plug in the monthly cost/gallon for gasoline once it's available.

In [26]:
# Define constants
vans_count = 1000
avg_miles_per_van = 10234

# Placeholder for monthly cost/gallon for gasoline (using a hypothetical value for demonstration)
gasoline_price_per_gallon = 3  # hypothetical value in USD

# Calculate total cost for gasoline for all vans for each month
gasoline_cost = vans_count * avg_miles_per_van * gasoline_price_per_gallon

# As we don't have varying gasoline prices for each month, the cost will remain consistent across months
monthly_gasoline_cost = pd.Series([gasoline_cost] * len(total_monthly_revenue), index=total_monthly_revenue.index)
monthly_gasoline_cost

2012-01-01    30702000
2012-02-01    30702000
2012-03-01    30702000
2012-04-01    30702000
2012-05-01    30702000
2012-06-01    30702000
2012-07-01    30702000
2012-08-01    30702000
2012-09-01    30702000
2012-10-01    30702000
2012-11-01    30702000
dtype: int64

The monthly gasoline cost for CW Water's fleet of delivery vans is calculated to be $30,702,000, given the hypothetical gasoline price of $3 per gallon.

## Other potential expenses involved in running the business:

Production Costs: The cost of producing the water bottles, including raw materials, labor, etc.



Marketing and Advertising: Any campaigns, promotions, or advertisements to promote the product.



Distribution & Warehousing: Storage costs, warehousing, and distribution to various retail points.



Salaries: Salaries for the staff, from the production line to the corporate office.



Maintenance: Maintenance of facilities, equipment, and delivery vans.




R&D: Research and development, especially if the company is looking into new products or improving existing ones.

##  create placeholders for these expenses :

In [27]:
# Hypothetical expenses (in USD)
expenses = {
    'Production Costs': 50000000,
    'Marketing and Advertising': 15000000,
    'Distribution & Warehousing': 10000000,
    'Salaries': 25000000,
    'Maintenance': 5000000,
    'R&D': 7000000
}

# Create a DataFrame with these hypothetical expenses for each month
monthly_expenses = pd.DataFrame(expenses, index=total_monthly_revenue.index)

# Calculate the total expenses for each month
monthly_expenses['Total Expenses'] = monthly_expenses.sum(axis=1)

monthly_expenses

,Production Costs,Marketing and Advertising,Distribution & Warehousing,Salaries,Maintenance,R&D,Total Expenses
2012-01-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-02-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-03-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-04-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-05-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-06-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-07-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-08-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-09-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000
2012-10-01,50000000,15000000,10000000,25000000,5000000,7000000,112000000


In [28]:
# Calculate Net Income for each month
net_income = total_monthly_revenue - monthly_expenses['Total Expenses'] - total_rebates - monthly_gasoline_cost

net_income

2012-01-01   1398634527.00000
2012-02-01    953038401.00000
2012-03-01   1514803077.00000
2012-04-01   1204961070.50000
2012-05-01   1663545613.00000
2012-06-01    966792719.00000
2012-07-01   1920031607.50000
2012-08-01    710632069.00000
2012-09-01   1196732499.50000
2012-10-01   1417942358.50000
2012-11-01   1663545613.00000
dtype: float64

some hypothetical monthly expenses for various categories are set up above, totaling $112,000,000 each month.

With revenues and expenses established, a simple income statement for each month can be drafted:

Net Income
=
Total Revenue
−
Total Expenses
−
Rebates
−
Gasoline Costs
Net Income=Total Revenue−Total Expenses−Rebates−Gasoline Costs

the next step is to calculate the net income for each month based on this formula.

In [29]:
# Calculate Net Income for each month
net_income = total_monthly_revenue - monthly_expenses['Total Expenses'] - total_rebates - monthly_gasoline_cost

net_income


2012-01-01   1398634527.00000
2012-02-01    953038401.00000
2012-03-01   1514803077.00000
2012-04-01   1204961070.50000
2012-05-01   1663545613.00000
2012-06-01    966792719.00000
2012-07-01   1920031607.50000
2012-08-01    710632069.00000
2012-09-01   1196732499.50000
2012-10-01   1417942358.50000
2012-11-01   1663545613.00000
dtype: float64

To summarize:

created a summary template of sales volumes by region and channel.


calculated the monthly seasonality of sales volumes.



calculated the total revenue for each month.



estimated rebates and other expenses.


computed the net income for each month.

